# Test

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

PATH = "/home/ubuntu/Archives/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

## All data

### Small/Large-gap data

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

# N = 17
# pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"

# final time for mean-field
T_final = 10000.

# number of points to get Lyapunov exponent for
npts = 256

# tolerance for DifferentialEquations.jl when solving mean-field 
tol = 1e-8;

In [ ]:
subdir = "small_gaps"
# subdir = "late_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
couplings_small_gap = Dict()
eigvals_small_gap = Dict()
lyapunov_exponents_small_gap = Dict()

for instance_name in instance_names
    seed = match(pattern, instance_name)[1]
    try    
        couplings = h5read(folder_name * instance_name, "J")
        couplings_small_gap[seed] = couplings
        eigvals_small_gap[seed] = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        lyapunov_exponents_small_gap[seed] = h5read(folder_name * instance_name, @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end
end


In [ ]:
subdir = "large_gaps"
# subdir = "extra_data"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)

couplings_large_gap = Dict()
eigvals_large_gap = Dict()
lyapunov_exponents_large_gap = Dict()

for instance_name in instance_names
    seed = match(pattern, instance_name)[1]
    try
        couplings = h5read(folder_name * instance_name, "J")
        couplings_large_gap[seed] = couplings
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        eigvals_large_gap[seed] = λ
        lyapunov_exponents_large_gap[seed] = h5read(folder_name * instance_name, @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts))    
    catch e
        printstyled(seed, ": ", e, "\n", color=:red)
    end                                             
end


In [ ]:
eigvals_small = [v[2, :] .- v[1, :] for (k, v) in eigvals_small_gap]
eigvals_large = [v[2, :] .- v[1, :] for (k, v) in eigvals_large_gap]
avg_eigvals_small = eigvals_small |> mean
avg_eigvals_large = eigvals_large |> mean;

avg_lyapunov_exponents_small = [v for (k, v) in lyapunov_exponents_small_gap] |> mean
avg_lyapunov_exponents_large = [v for (k, v) in lyapunov_exponents_large_gap] |> mean;

In [ ]:
figure(figsize=(7, 3))
subplot(121)
plot(range(0, 1, 33), avg_eigvals_small, "-o", ms=3)
plot(range(0, 1, 33), avg_eigvals_large, "-s", ms=3)
xlim(0, 1)
ylim(0, 2)
xlabel("\$s\$")
ylabel("Average Exact Gap")

subplot(122)
plot(range(0, 1, npts), avg_lyapunov_exponents_small, "-", ms=3)
plot(range(0, 1, npts), avg_lyapunov_exponents_large, "-", ms=3)
xlim(0, 1)
ylim(0, 0.5)
xlabel("\$s\$")
ylabel("Average \$\\lambda\$")
tight_layout()

In [ ]:
length(couplings_small_gap) |> println
length(couplings_large_gap) |> println

In [ ]:
gapsize_small_gap = []
gaplocs_small_gap = []
for (k, v) in eigvals_small_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_small_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_small_gap, gaploc)
end 

In [ ]:
gapsize_large_gap = []
gaplocs_large_gap = []
for (k, v) in eigvals_large_gap
    gap = v[2, :] .- v[1, :]
    minigap = minimum(gap)
    push!(gapsize_large_gap, minigap)
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minigap, gap)]
    push!(gaplocs_large_gap, gaploc)
end 

In [ ]:
figure(figsize=(7, 3))
ax = subplot(121)
hist2D(gapsize_small_gap, gaplocs_small_gap, bins=32, range=[[0, 1e-2], [0., 1.0]], cmap="gist_earth_r")
colorbar()
xlim(0, 0.01)
ylim(0., 1)
xticks([0., 5e-3, 1e-2])
ax.set_xticklabels([0., "", 1e-2])
xlabel("\$\\Delta\$")
ylabel("\$s_{\\mathrm{min}}\$")

ax = subplot(122)
hist2D(gapsize_large_gap, gaplocs_large_gap, bins=32, range=[[0.5, 1.1], [0., 1.0]], cmap="gist_earth_r")
colorbar()
xlim(0.5, 1.1)
ylim(0.0, 1)
ax.set_yticklabels([])
xlabel("\$\\Delta\$")
# ylabel("\$s_{\\mathrm{min}}\$")
tight_layout()

## Single instance

In [ ]:
# N = 9
# seed = "6"
seed = "42"
# seed = "3017"
# seed = "6150"
seed = "11068"

# N = 15
# seed = "36"
# seed = "325"
# seed = "19015"
# seed = "19134"
# seed = "20484"
J_mat = couplings_small_gap[seed];
λ = eigvals_small_gap[seed];

# seed = "2015"
# J_mat = couplings_large_gap[seed];
# λ = eigvals_large_gap[seed];

In [ ]:
mf_problem = Problem(0, J_mat);

### Exact gap

In [ ]:
exact_times = range(0, 1, 33);

In [ ]:
figure(figsize=(4, 3))
subplot(111)
for i in 1:size(λ)[1]
    plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", ms=2)
end
xlim(0., 1.)
ylim(0, 2.)
xlabel("\$s\$")
ylabel("Exact Eigenvalues")
tight_layout()

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap) 
mingap |> println
gaploc = exact_times[findfirst(x -> x == minimum(gap), gap)] 
gaploc |> println

In [ ]:
bogo_spec = bogoliubov_spectrum(mf_problem, LyapunovParameters(64000, 32, tol, tol))
bogo_spec = reduce(hcat, bogo_spec)
bogo_spec = sort(bogo_spec .|> real, dims=1);

In [ ]:
figure(figsize=(4, 3))
subplot(111)
for i in 1:size(bogo_spec)[1]
    # plot(exact_times[2:end], bogo_spec[i, :], "-k", ms=2)
    # plot(exact_times, (λ[i, :] .- λ[1, :]), "--r", ms=2)
    plot(exact_times[2:end], bogo_spec[i, :] ./ 2pi, "--r", ms=2)
    plot(exact_times, (λ[i, :] .- λ[1, :]) ./ 2pi, "-k", ms=2)    
end
xlim(0., 1.)
ylim(-0., 0.5)
xlabel("\$s\$")
ylabel("Exact Eigenvalues")
tight_layout()

### Mean-field trajectories

In [ ]:
# T_final = 1000. 
# T_final = 4000. 
# T_final = 8000. 
# T_final = 16000.
T_final = 32000.
# T_final = 64000.
tol = 1e-8;

In [ ]:
sol = evolve_mean_field(mf_problem.local_fields, mf_problem.couplings, T_final, t -> t/T_final, rtol=1e2*tol, atol=tol)
size(sol.t)[1] 

In [ ]:
component_dict = Dict("x" => 1, "y" => 2, "z" => 3)
component = "z"
test_data = reduce(hcat, [sol.u[k][component_dict[component], :] for k in 1:size(sol.u)[1]]);

In [ ]:
figure(figsize=(4, 3))

subplot(111)
for spin_nr in 1:N-1
    if round(abs.(test_data[spin_nr, :][end]), digits=3) == 1.0
        plot(sol.t ./ T_final, test_data[spin_nr, :], "-")
    else
        plot(sol.t ./ T_final, test_data[spin_nr, :], "-", label=@sprintf("\$i=%i\$", spin_nr))    
    end
end

xlim(0, 1)
ylim(-1, 1)
xlabel("\$s\$")
ylabel("\$n_i^z(s)\$")
legend(frameon=false)

tight_layout()
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")

### Statistical Green function

In [ ]:
tol = 1e-8
# T_final = 32000.
npts = 2048
coarse_times = range(0, 1, npts + 1);

In [ ]:
lyapunov_parameters = LyapunovParameters(T_final, npts, tol, tol)
mf_sol, stat_GF = statistical_green_function(mf_problem, lyapunov_parameters);

In [ ]:
flucs = k -> (real.(1.0im .* diag(stat_GF[k])[1:mf_problem.num_qubits]) .- 1.0) ./ 2;
all_flucs = reduce(hcat, map(flucs, 1:npts+1));

In [ ]:
figure(figsize=(20, 4))
ylims = 0.25
for i in 1:(N-1)÷2
    ax = subplot(2, (N-1)÷2, i)
    plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
    # axvline(coarse_times[findfirst(x -> x == maximum(all_flucs[i, :]), all_flucs[i, :])], c="r")
    xlim(0., 1.)
    ax.set_xticklabels([])
    ylim(0., ylims)
    if i > 1
        ax.set_yticklabels([])
    end
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

for i in (N-1)÷2+1:N-1
    ax = subplot(2, (N-1)÷2, i)
    plot(coarse_times, all_flucs[i, :], label=@sprintf("\$i=%s\$", string(i)))
#     println(i, ": ", maximum(all_flucs[i, :]))
    # axvline(coarse_times[findfirst(x -> x == maximum(all_flucs[i, :]), all_flucs[i, :])], c="r")    
    xlim(0., 1.)
    ylim(0., ylims)
    if i > (N-1)÷2+1
        ax.set_yticklabels([])
    end    
    xlabel("\$s\$")    
    legend(frameon=false, ncol=2, handlelength=0)
end

ax = subplot(2, (N-1)÷2, (N-1)÷2 + 1)
ax.set_ylabel("\$F_{ii}(t, t)\$")

tight_layout()
# savefig("../plots/" * @sprintf("fluctuations_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")

### Spectra

In [ ]:
# npts_diag = 16
# τ_final = 1500.
# spectral_sols = evolve_spectral_function(mf_problem, T_final, τ_final, npts_diag=npts_diag, rtol=1e-6, atol=1e-6);

In [ ]:
npts_diag = 10
T_diags = T_final .* range(0.6, 0.9, npts_diag+1)
T_diags = T_final .* [0.75, 0.85]
τ_final = 2000.
spectral_sols = evolve_spectral_function(mf_problem, T_final, τ_final, T_diags, rtol=1e-6, atol=1e-6);

In [ ]:
spec_sum = []
for k in 1:length(T_diags)
    push!(spec_sum, spectral_sum(spectral_sols[k]))
end

In [ ]:
spectral_data = Dict();

In [ ]:
for spin_idx in 1:N-1
    spectral_data[spin_idx] = spectrum_vs_T(spin_idx, spectral_sols)
end

In [ ]:
spin_idx = 6

# map data to functions
spectral_fs = []
for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))
    push!(spectral_fs, linear_interpolation(ωs, spec, extrapolation_bc=Line()))
end

In [ ]:
# spin_idx = 2
num_plots = length(spectral_sols)
xlim_vals = (-0.05, 0.3)
# fig, axs = subplots(figsize=(5, 12), nrows=num_plots, ncols=1)
fig, axs = subplots(figsize=(5, 5), nrows=num_plots, ncols=1)

# T_diags = range(0, 1, npts_diag+1)
for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))
    ax = axs[k] # subplot(num_plots, 1, k)     

    ax.plot(spec_sum[k]..., "-C2")

    ax.plot(ωs, spec, "-", label=@sprintf("\$T=%0.3f\$", T_diags[k] ./ T_final))
    ax.axvline(mingap, ls="--", c="k", alpha=0.2)
    # max_loc = ωs[findfirst(x->x==maximum(spec), spec)]
    # axvline.([max_loc], c="r", ls="--")

    ax.set_xlim(xlim_vals...)
    ax.set_ylim(0, 0.1)  
    ax.set_xticks(range(xlim_vals..., 8))  
    ax.set_xticklabels([])
    # ax.set_ylabel(@sprintf("\$\\rho_{%i%i}(T, \\omega)\$", idx, idx))
    ax.legend(frameon=false, handlelength=0, fontsize=12)
    if k%2 == 0
        ax.yaxis.tick_right()
    end
    ax.tick_params(axis="y", labelsize=10)
end
ax = subplot(num_plots, 1, num_plots)
ax.set_xlabel("\$\\nu\$")
ax.set_xticklabels(round.(range(xlim_vals..., 8), digits=2))

fig.text(0.0, 0.5, @sprintf("\$\\rho_{%i%i}(T, \\omega)\$", spin_idx, spin_idx), va="center", rotation="vertical")
subplots_adjust(wspace=0.5, hspace=0.16)
# savefig("../plots/" * @sprintf("spectrum_spin_%02i_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", spin_idx, idx, N, num_clauses), dpi=256, bbox_inches="tight")

In [ ]:
omega_range = range(0.0 , 0.3, 1001) |> collect
data_2D = [map(spectral_fs[k], omega_range) for (k, (ωs, spec)) in enumerate(zip(spectral_data[spin_idx]...))];

In [ ]:
cmap = "gist_yarg"

figure(figsize=(4, 3))
Y, X = meshgrid(omega_range, T_diags ./ T_final);
ax = subplot(111)
heatmap = ax.pcolormesh(Y, X, data_2D, cmap=cmap, rasterized=true, vmin=0, vmax=1)
heatmap.set_edgecolor("face")
# ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
# cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\omega\$")
ax.set_ylabel("\$T\$")
ax.set_xlim(omega_range[1], omega_range[end])
ax.set_ylim(T_diags[1] ./ T_final, T_diags[end] ./ T_final)
# ax.set_xticks(t_scale .* [0, T/2, T])
# ax.set_yticks(t_scale .* [0, T/2, T])


tight_layout(pad=0.75, w_pad=0.25, h_pad=0)

In [ ]:
data_2D_f = interpolate((omega_range, T_diags), reduce(hcat, data_2D), Gridded(Linear()));
interp_data_2D = [[data_2D_f(ω, T) for ω in omega_range] for T in range(T_diags[1], T_diags[end], 1001)];

In [ ]:
figure(figsize=(4, 3))
Y, X = meshgrid(omega_range, range(T_diags[1], T_diags[end], 1001) ./ T_final);
ax = subplot(111)
heatmap = ax.pcolormesh(Y, X, interp_data_2D, cmap=cmap, rasterized=true, vmin=0, vmax=1)
heatmap.set_edgecolor("face")
# ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
ax.set_xlim(omega_range[1], omega_range[end])
# ax.set_ylim(0, 1)
# ax.set_xticks(t_scale .* [0, T/2, T])
# ax.set_yticks(t_scale .* [0, T/2, T])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)

## Scripted spectra

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

subdir = "small_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
T_final = 16000.
T_final = 32000.
τ_final = 1000.
τ_final = 2000.
# τ_final = 4000.
all_ωs = []
spec_sum = []

num_plots = 100
num_plots = 141
for instance_name in instance_names[1:num_plots]
    try
        
        seed = match(pattern, instance_name)[1]    

        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")

        gap = λ[2, :] .- λ[1, :];
        exact_times = range(0, 1, 33)
        gaploc = exact_times[findfirst(x -> x == minimum(gap), gap)] 
        T_diags = T_final .* [gaploc - 0.1, gaploc - 0.05, gaploc]
        
        # for k in 1:1
        # for k in 2:2
        for k in 3:3
            push!(all_ωs, h5read(folder_name * instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.3f/omegas", T_final, τ_final, T_diags[k] / T_final)))
            push!(spec_sum, h5read(folder_name * instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.3f/data", T_final, τ_final, T_diags[k] / T_final)))
        end   
    catch
        printstyled(instance_name, "\n", color=:red)
    end
end

In [ ]:
num_plots = length(spec_sum)

In [ ]:
# map data to functions
spectral_sum_fs = []
for (k, (ωs, spec)) in enumerate(zip(all_ωs, spec_sum))
    push!(spectral_sum_fs, linear_interpolation(ωs, spec, extrapolation_bc=Line()))
end

In [ ]:
xlim_vals = (-0.05, 0.2)
new_ωs = range(xlim_vals..., 2001)
spectral_average = mean([map(spectral_sum_fs[k], new_ωs) for k in 1:length(spectral_sum_fs)]);

In [ ]:
figure(figsize=(4, 3))
subplot(111)
plot(new_ωs, spectral_average, "-k", ms=2)    
xlim(xlim_vals...)
ylim(-0., 0.1)
xlabel("\$\\nu\$")
ylabel("Ensemble-averaged spectrum", fontsize=12)
tight_layout()

In [ ]:
xlim_vals = (-0.05, 0.1)
fig, axs = subplots(figsize=(5, 50), nrows=num_plots, ncols=1)

for (k, (ωs, spec)) in enumerate(zip(all_ωs, spec_sum))
    ax = axs[k]     
    ax.plot(ωs, spec, "-")

    ax.set_xlim(xlim_vals...)
    ax.set_ylim(0, 0.05)  
    ax.set_xticks(range(xlim_vals..., 7))  
    ax.set_xticklabels([])
    # ax.set_ylabel(@sprintf("\$\\rho_{%i%i}(T, \\omega)\$", idx, idx))
    # ax.legend(frameon=false, handlelength=0, fontsize=12)
    if k%2 == 0
        ax.yaxis.tick_right()
    end
    ax.tick_params(axis="y", labelsize=10)
end
ax = subplot(num_plots, 1, num_plots)
ax.set_xlabel("\$\\nu\$")
ax.set_xticklabels(round.(range(xlim_vals..., 7), digits=3))

subplots_adjust(wspace=0.5, hspace=0.16)

### Delete data...

In [ ]:
for instance_name in instance_names#[1:110]
    delfile = folder_name * instance_name
    seed = match(pattern, instance_name)[1]    
    if seed == "52411"
        println(seed)
        h5open(delfile, "r+") do f
            key = @sprintf("lyapunov_exponent_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), npts) 
            println(key)       
            if haskey(f, key)
                printstyled(@sprintf("key %s", key) * "\n", color=:red)
                # delete_object(f, key)
            end
        end
    end
end

In [ ]:
for instance_name in instance_names[1:110]
    delfile = folder_name * instance_name
    seed = match(pattern, instance_name)[1]    

    λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")

    gap = λ[2, :] .- λ[1, :];
    exact_times = range(0, 1, 33)
    gaploc = exact_times[findfirst(x -> x == minimum(gap), gap)] 
    T_diags = T_final .* [gaploc - 0.05, gaploc]  

    h5open(delfile, "r+") do f
        key = "bogoliubov_spectrum"
        key = "fluctuations"
        # key = @sprintf("spectra/T_%0.3f/omegas", T_diags[1] / T_final)
        # key = @sprintf("spectra/T_%0.3f/data", T_diags[1] / T_final)
        # key = @sprintf("spectra/T_%0.3f/omegas", T_diags[2] / T_final)
        # key = @sprintf("spectra/T_%0.3f/data", T_diags[2] / T_final)        
        if haskey(f, key)
            printstyled(@sprintf("key %s", key) * "\n", color=:red)
            # delete_object(f, key)
        end
        
    end
end

## Data admin

In [ ]:
N = 17
folder_name = PATH * @sprintf("data/SK_model/N_%i/", N)
instance_names = readdir(folder_name)

In [ ]:
minigaps = Dict()
for instance_name in instance_names
    try
        pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"
        seed = match(pattern, instance_name)[1]    

        # couplings = h5read(folder_name * instance_name, "J", J_mat)
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        minigap = minimum(λ[2, :] .- λ[1, :])
        minigaps[seed] = minigap
    catch
        print(instance_name, ", ")
    end
end


In [ ]:
sorted_minigaps = sort(collect(minigaps), by=x->x[2])
length(sorted_minigaps)

In [ ]:
first_large_gap_idx = findfirst(x -> x > 0.1, [x for (key, x) in sorted_minigaps])

In [ ]:
small_minigaps = sorted_minigaps[1:first_large_gap_idx-1]
large_minigaps = sorted_minigaps[first_large_gap_idx:end];

In [ ]:
large_minigaps

#### Move data

In [ ]:
# # mv_list = [k for (k, v) in small_minigaps]
# mv_list = [k for (k, v) in large_minigaps]

# type_str = "data"
# file_suffix = "h5"

# for seed in mv_list
#     try
#         # mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "small_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
#         mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "large_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
#     catch
#         print(seed, ", ")
#     end
# end

#### Move plots

In [ ]:
# # mv_list = [k for (k, v) in small_minigaps]
# mv_list = [k for (k, v) in large_minigaps]

# type_str = "plots"
# file_suffix = "pdf"

# for seed in mv_list
#     try
#         # mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "small_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
#         mv(PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix), PATH * @sprintf("%s/SK_model/N_%i/", type_str, N) * "large_gaps/" * @sprintf("random_SK_instance_N_%i_seed_%s.%s", N, seed, file_suffix))
#     catch
#         print(seed, ", ")
#     end
# end

#### Late gaps...

In [ ]:
# N = 15
# all_late_gap_seeds = []
# for screen in 1:5000:45000
#     late_gap_seeds = h5read(PATH * @sprintf("data/SK_model/N_%i/", N) * @sprintf("random_SK_instances_N_%i_screen_%i_late_minigaps.h5", N, screen), "late_gap_seeds")
#     push!(all_late_gap_seeds, late_gap_seeds)
# end
# all_late_gap_seeds = reduce(vcat, all_late_gap_seeds)
# # h5write(PATH * @sprintf("data/SK_model/N_%i/", N) *@sprintf("random_SK_instances_N_%i_late_minigaps.h5", N), "late_gap_seeds", all_late_gap_seeds);

In [ ]:
N = 17
h5read(PATH * @sprintf("data/SK_model/N_%i/", N) *@sprintf("random_SK_instances_N_%i_late_minigaps.h5", N), "late_gap_seeds") # |> length

In [ ]:
# all_late_gap_seeds_N_17 = [15001, 17004, 3005, 9006, 4008, 16008, 11009, 15010, 10012, 9014, 15020, 22, 2028, 12027, 14029, 17035, 1034, 9037, 2041, 37, 14039, 8039, 10041, 3040, 8040, 15046, 10045, 15049, 11046, 12047, 13048, 18053, 7051, 20052, 8057, 4059, 9060, 14060, 16064, 18064, 13061, 13062, 18066, 66, 5065, 14065, 6066, 74, 17077, 14076, 19083, 15084, 8082, 17085, 12084, 8087, 15096, 20087, 14091, 12091, 3091, 13093, 17101, 8103, 19110, 2117, 2121, 10109, 16117, 1119, 5119, 5121, 4119, 6117, 122, 17120, 18125, 4123, 16138, 12131, 3129, 19140, 8135, 19141, 6136, 140, 5142, 14138, 16148, 145, 4144, 9154, 2157, 5154, 5158, 10153, 2171, 8165, 12163, 4169, 12166, 19181, 5178, 12175, 4178, 16187, 10173, 20174, 11188, 14181, 10183, 17185, 3186, 17188, 16205, 19207, 9210, 5205, 209, 18207, 3199, 8210, 15232, 13213, 4224, 17221, 11233, 233, 11235, 19240, 18232, 15246]
# h5write(PATH * @sprintf("data/SK_model/N_%i/", 17) *@sprintf("random_SK_instances_N_%i_late_minigaps.h5", 17), "late_gap_seeds", sort(all_late_gap_seeds_N_17));

In [ ]:
minigap_locs = []
for (k, v) in eigvals_small_gap
    minigap = minimum(v[2, :] .- v[1, :])
    push!(minigap_locs, findfirst(x -> x == minigap, v[2, :] .- v[1, :]))
end

In [ ]:
findall(x -> x != 30, minigap_locs)